In [41]:
import pandas as pd
import datetime as dt

In [16]:
file = 'logins.json'
logins = pd.read_json(file)
logins.head()

,login_time
0,1970-01-01 20:13:18
1,1970-01-01 20:16:10
2,1970-01-01 20:16:37
3,1970-01-01 20:16:36
4,1970-01-01 20:26:21


In [17]:
logins['15_minute_intervals'] = [i*15 for i in range(logins.size)]
logins.head()

,login_time,15_minute_intervals
0,1970-01-01 20:13:18,0
1,1970-01-01 20:16:10,15
2,1970-01-01 20:16:37,30
3,1970-01-01 20:16:36,45
4,1970-01-01 20:26:21,60


In [61]:
logins['base_time'] = dt.datetime(logins['login_time'].dt.year[0],
                                  logins['login_time'].dt.month[0],
                                  logins['login_time'].dt.day[0],
                                  logins['login_time'].dt.hour[0],
                                  minute=0)
logins.head()

,login_time,15_minute_intervals,base_time
0,1970-01-01 20:13:18,0,1970-01-01 20:00:00
1,1970-01-01 20:16:10,15,1970-01-01 20:00:00
2,1970-01-01 20:16:37,30,1970-01-01 20:00:00
3,1970-01-01 20:16:36,45,1970-01-01 20:00:00
4,1970-01-01 20:26:21,60,1970-01-01 20:00:00


In [63]:
logins['interval_start'] = logins['base_time'] + pd.TimedeltaIndex(logins['15_minute_intervals'], unit='m')
logins.head()

,login_time,15_minute_intervals,base_time,interval_start
0,1970-01-01 20:13:18,0,1970-01-01 20:00:00,1970-01-01 20:00:00
1,1970-01-01 20:16:10,15,1970-01-01 20:00:00,1970-01-01 20:15:00
2,1970-01-01 20:16:37,30,1970-01-01 20:00:00,1970-01-01 20:30:00
3,1970-01-01 20:16:36,45,1970-01-01 20:00:00,1970-01-01 20:45:00
4,1970-01-01 20:26:21,60,1970-01-01 20:00:00,1970-01-01 21:00:00
